# 使用领域（私有）数据微调 ChatGLM3

生成带有 epoch 和 timestamp 的模型文件

In [1]:
import torch

print(torch.__config__.show(), torch.cuda.get_device_properties(0))

PyTorch built with:
  - C++ Version: 199711
  - MSVC 192930151
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.8.1  (built against CUDA 12.0)
  - Magma 2.5.4
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.8.1, CXX_COMPILER=C:/cb/pytorch_1000000000000/work/tmp_bin/sccache-cl.exe, CXX_FLAGS=/DWIN32 /D_WINDOWS /GR /EHsc /bigobj /FS -DUSE_PTHREADPOOL -DNDEBUG -DUSE_

In [2]:
# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
# train_data_path = 'data/zhouyi_dataset_handmade.csv'    # 训练数据路径
# train_data_path = 'data/zhouyi_dataset_20240118_152413.csv'
train_data_path = 'data/zhouyi_dataset_20240118_163659.csv'    # 训练数据路径(批量生成数据集）
eval_data_path = None                     # 验证数据路径，如果没有则设置为None
seed = 8                                 # 随机种子
max_input_length = 512                    # 输入的最大长度
max_output_length = 1536                  # 输出的最大长度
lora_rank = 16                             # LoRA秩
lora_alpha = 32                           # LoRA alpha值
lora_dropout = 0.05                       # LoRA Dropout率
prompt_text = ''                          # 所有数据前的指令文本

## 数据处理

In [3]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files=train_data_path)
print(dataset)

d:\anaconda3\envs\chatglm3-demo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 0 examples [00:00, ? examples/s]d:\anaconda3\envs\chatglm3-demo\lib\site-packages\datasets\download\streaming_download_manager.py:778: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
Generating train split: 160 examples [00:00, 25990.03 examples/s]

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 160
    })
})


In [4]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [5]:
show_random_elements(dataset["train"], num_examples=5)

,content,summary
0,师卦和教育启蒙有什么联系？,师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表军队和指挥军情的卦象。根据《象辞》，这一卦象被解释为“地中有水”，象征着像大地一样包容和养育众人。根据《断易天机》，只有德高望重的长者来统率军队，才能获得吉祥无咎。\n\n\n据北宋易学家邵雍解，得师卦者将面临困难重重，忧心劳众，宜包容别人，艰苦努力，摒除一切困难。台湾国学大儒傅佩荣解则提到，对于时运、财运、家宅和身体等方面会有相应影响。\n\n\n传统解卦认为，师卦具有养兵聚众、出师攻伐之象，彼此有伤，难得安宁的大象。在运势方面，预示着困难重重，需要以正规行事，谨小慎微，严于律已。在事业、经商、求名、婚恋和决策等方面，都需要保持冷静、谨慎，注意避免敌人和困难带来的不利影响，必能成功。
1,周易师卦的深层含义是什么？,师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表军队和指挥军情的卦象。根据《象辞》，这一卦象被解释为“地中有水”，象征着像大地一样包容和养育众人。根据《断易天机》，只有德高望重的长者来统率军队，才能获得吉祥无咎。\n\n\n据北宋易学家邵雍解，得师卦者将面临困难重重，忧心劳众，宜包容别人，艰苦努力，摒除一切困难。台湾国学大儒傅佩荣解则提到，对于时运、财运、家宅和身体等方面会有相应影响。\n\n\n传统解卦认为，师卦具有养兵聚众、出师攻伐之象，彼此有伤，难得安宁的大象。在运势方面，预示着困难重重，需要以正规行事，谨小慎微，严于律已。在事业、经商、求名、婚恋和决策等方面，都需要保持冷静、谨慎，注意避免敌人和困难带来的不利影响，必能成功。
2,周易中坤卦的解释是什么？,"坤卦是周易中的一卦，代表大地的形势，象征顺顺利利，顺应天道。它由两个坤卦叠加而成，全为阴卦，具有纯阴性质。在卜问中，坤卦预示着利于雌马的贞正之象，吉利的事情会发生。同时，出行需谨慎，起初可能会迷失方向，但最终会找到主人，带来吉利。前往西南会得到朋友，前往东北会失去朋友。在安定的状态下，能够得到吉祥。\n\n坤卦的形势平铺舒展，代表地道生育抚养万物，依循天时地利，象征以厚德载物。在决策上，应该顺从运势，守正静待，不宜急进，须以静制动为宜。事业发展上，要注重内心修养，广纳众意，以柔克刚为原则，力求和谐共生。经商求名上，同样要稳健行事，不宜冒险急进，在合作中共同完成事业。\n\n综上所述，坤卦意味着安稳、柔顺、温和、顺应天道，需要以厚德载物，以守正静待为策略，以诚信待人，收敛于己，力求温和和谐。""\n\nformat:""卦名:坤卦\n解释:坤卦是周易中的一卦，代表大地的形势，象征顺顺利利，顺应天道。它由两个坤卦叠加而成，全为阴卦，具有纯阴性质。在卜问中，坤卦预示着利于雌马的贞正之象，吉利的事情会发生。同时，出行需谨慎，起初可能会迷失方向，但最终会找到主人，带来吉利。前往西南会得到朋友，前往东北会失去朋友。在安定的状态下，能够得到吉祥。\n\n坤卦的形势平铺舒展，代表地道生育抚养万物，依循天时地利，象征以厚德载物。在决策上，应该顺从运势，守正静待，不宜急进，须以静制动为宜。事业发展上，要注重内心修养，广纳众意，以柔克刚为原则，力求和谐共生。经商求名上，同样要稳健行事，不宜冒险急进，在合作中共同完成事业。\n\n综上所述，坤卦意味着安稳、柔顺、温和、顺应天道，需要以厚德载物，以守正静待为策略，以诚信待人，收敛于己，力求温和和谐。"
3,周易的坤卦讲述了什么？,"坤卦是周易中的一卦，代表大地的形势，象征顺顺利利，顺应天道。它由两个坤卦叠加而成，全为阴卦，具有纯阴性质。在卜问中，坤卦预示着利于雌马的贞正之象，吉利的事情会发生。同时，出行需谨慎，起初可能会迷失方向，但最终会找到主人，带来吉利。前往西南会得到朋友，前往东北会失去朋友。在安定的状态下，能够得到吉祥。\n\n坤卦的形势平铺舒展，代表地道生育抚养万物，依循天时地利，象征以厚德载物。在决策上，应该顺从运势，守正静待，不宜急进，须以静制动为宜。事业发展上，要注重内心修养，广纳众意，以柔克刚为原则，力求和谐共生。经商求名上，同样要稳健行事，不宜冒险急进，在合作中共同完成事业。\n\n综上所述，坤卦意味着安稳、柔顺、温和、顺应天道，需要以厚德载物，以守正静待为策略，以诚信待人，收敛于己，力求温和和谐。""\n\nformat:""卦名:坤卦\n解释:坤卦是周易中的一卦，代表大地的形势，象征顺顺利利，顺应天道。它由两个坤卦叠加而成，全为阴卦，具有纯阴性质。在卜问中，坤卦预示着利于雌马的贞正之象，吉利的事情会发生。同时，出行需谨慎，起初可能会迷失方向，但最终会找到主人，带来吉利。前往西南会得到朋友，前往东北会失去朋友。在安定的状态下，能够得到吉祥。\n\n坤卦的形势平铺舒展，代表地道生育抚养万物，依循天时地利，象征以厚德载物。在决策上，应该顺从运势，守正静待，不宜急进，须以静制动为宜。事业发展上，要注重内心修养，广纳众意，以柔克刚为原则，力求和谐共生。经商求名上，同样要稳健行事，不宜冒险急进，在合作中共同完成事业。\n\n综上所述，坤卦意味着安稳、柔顺、温和、顺应天道，需要以厚德载物，以守正静待为策略，以诚信待人，收敛于己，力求温和和谐。"
4,比卦在周易中怎样表达教育的概念？,"比卦""是周易卦象中的一枚卦，由下卦坤（地）上卦坎（水）组成，预示着吉利的变化。在卜筮时，再次占卜依然吉利，预示长期稳定无灾祸。然而，当不愿臣服的邦国未能前来朝贺时，将会带来危险。\n\n在《象辞》中，比卦被描述为地上有水的情景，反映了相亲相依相互依赖的意义。先王观此卦象，取法于水附大地，地纳江河之象，因此此卦被解释为建立万国，亲近诸侯。\n\n北宋易学家邵雍解释认为，比卦代表水在地面上流动，人际关系亲密和睦，各种事情无忧无虑。\n\n台湾国学大儒傅佩荣解释认为，比卦的时运是众人相贺，财运是善人相扶，家宅方面是长久美满，身体方面则需早求治心腹水肿。\n\n传统解卦认为，比卦预示着相亲相辅，宽宏无私，精诚团结的道理。在运势上，表示平顺，能得贵人提拔，事业宜速战速决，不宜过度迟疑。对于事业、经商、求名、婚恋等方面都有积极的影响，提醒人们待人宽厚、正直，主动热情，并谨慎选择朋友。\n\n总之，""比卦"


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

d:\anaconda3\envs\chatglm3-demo\lib\site-packages\transformers\utils\hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [7]:
# tokenize_func 函数
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """

    # 构建问题文本
    question = prompt_text + example['content']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['summary']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}


In [8]:
column_names = dataset['train'].column_names
tokenized_dataset = dataset['train'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False, 
    remove_columns=column_names
)

Map: 100%|██████████| 160/160 [00:00<00:00, 1552.35 examples/s]


In [9]:
tokenized_dataset = tokenized_dataset.shuffle(seed=seed)
tokenized_dataset = tokenized_dataset.flatten_indices()

Flattening the indices:   0%|          | 0/160 [00:00<?, ? examples/s]

Flattening the indices: 100%|██████████| 160/160 [00:00<00:00, 25381.57 examples/s]


In [10]:
import torch
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:
    """
    用于处理批量数据的DataCollator，尤其是在使用 ChatGLM 模型时。

    该类负责将多个数据样本（tokenized input）合并为一个批量，并在必要时进行填充(padding)。

    属性:
    pad_token_id (int): 用于填充(padding)的token ID。
    max_length (int): 单个批量数据的最大长度限制。
    ignore_label_id (int): 在标签中用于填充的ID。
    """

    def __init__(self, pad_token_id: int, max_length: int = 2048, ignore_label_id: int = -100):
        """
        初始化DataCollator。

        参数:
        pad_token_id (int): 用于填充(padding)的token ID。
        max_length (int): 单个批量数据的最大长度限制。
        ignore_label_id (int): 在标签中用于填充的ID，默认为-100。
        """
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """
        处理批量数据。

        参数:
        batch_data (List[Dict[str, List]]): 包含多个样本的字典列表。

        返回:
        Dict[str, torch.Tensor]: 包含处理后的批量数据的字典。
        """
        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)  # 找到最长的样本长度

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d  # 计算需要填充的长度
            # 添加填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))

        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)

        return {'input_ids': input_ids, 'labels': labels}


In [11]:
# 准备数据整理器
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)

## 加载模型

In [12]:
from transformers import AutoModel, BitsAndBytesConfig

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

model.supports_gradient_checkpointing = True  
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- modeling_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


bin d:\anaconda3\envs\chatglm3-demo\lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Loading checkpoint shards: 100%|██████████| 7/7 [00:05<00:00,  1.36it/s]
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [13]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

kbit_model = prepare_model_for_kbit_training(model)
target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [14]:
target_modules

['query_key_value']

In [15]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [16]:
qlora_model = get_peft_model(kbit_model, lora_config)
qlora_model.print_trainable_parameters()

trainable params: 3,899,392 || all params: 6,247,483,392 || trainable%: 0.06241540401681151


### QLoRA 微调模型

In [17]:
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

train_epochs = 3
output_dir = f"models/{model_name_or_path}-epoch{train_epochs}-{timestamp}"

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,                            # 输出目录
    per_device_train_batch_size=8,                     # 每个设备的训练批量大小
    gradient_accumulation_steps=1,                     # 梯度累积步数
    learning_rate=1e-3,                                # 学习率
    num_train_epochs=train_epochs,                     # 训练轮数
    lr_scheduler_type="linear",                        # 学习率调度器类型
    warmup_ratio=0.1,                                  # 预热比例
    logging_steps=1,                                 # 日志记录步数
    save_strategy="steps",                             # 模型保存策略
    save_steps=10,                                    # 模型保存步数
    optim="adamw_torch",                               # 优化器类型
    fp16=True,                                        # 是否使用混合精度训练
)


In [19]:
trainer = Trainer(
        model=qlora_model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator
    )

In [20]:
trainer.train()

  0%|          | 0/60 [00:00<?, ?it/s]d:\anaconda3\envs\chatglm3-demo\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
  2%|▏         | 1/60 [00:03<03:24,  3.46s/it]

{'loss': 3.594, 'learning_rate': 0.00016666666666666666, 'epoch': 0.05}


  3%|▎         | 2/60 [00:05<02:37,  2.72s/it]

{'loss': 4.0491, 'learning_rate': 0.0003333333333333333, 'epoch': 0.1}


  5%|▌         | 3/60 [00:11<04:00,  4.21s/it]

...

100%|██████████| 60/60 [04:47<00:00,  5.13s/it]

{'loss': 0.0033, 'learning_rate': 0.0, 'epoch': 3.0}


100%|██████████| 60/60 [04:48<00:00,  4.80s/it]

{'train_runtime': 288.208, 'train_samples_per_second': 1.665, 'train_steps_per_second': 0.208, 'train_loss': 0.6969813761301339, 'epoch': 3.0}


TrainOutput(global_step=60, training_loss=0.6969813761301339, metrics={'train_runtime': 288.208, 'train_samples_per_second': 1.665, 'train_steps_per_second': 0.208, 'train_loss': 0.6969813761301339, 'epoch': 3.0})

In [21]:
trainer.model.save_pretrained(output_dir)